In [26]:
import pandas as pd

In [27]:
# Charger les deux fichiers CSV
silver_I = pd.read_csv('data/silver_I.csv')
silver_II = pd.read_csv('data/silver_II.csv')

# Concaténer les deux DataFrames
df = pd.concat([silver_I, silver_II], ignore_index=True)


In [28]:
df = df[df['Customer ID'] != 0]

In [29]:
#Suppression de l'écriture scientifique
pd.options.display.float_format = '{:20,.2f}'.format
df.describe()

,Quantity,Price,Customer ID,Total,Order_Quantity,Order_Total,Order_count,Cancelled_order_count,Percent_cancelled,Average_Quantity,Average_Price,Average_Basket,StockCode_count,Recency
count,"797,885.00","797,885.00","797,885.00","797,885.00","797,885.00","797,885.00","797,885.00","797,885.00","797,885.00","797,885.00","797,885.00","797,885.00","797,885.00","797,885.00"
mean,12.60,3.70,"15,313.06",20.42,413.77,683.28,41.82,0.29,0.70,12.60,3.70,20.42,343.56,340.44
std,191.67,71.39,"1,696.47",313.52,"1,567.14","1,412.78",84.18,3.64,5.02,45.61,46.71,311.29,467.65,220.76
min,"-80,995.00",0.00,"12,346.00","-168,469.60","-87,167.00","-168,469.60",1.00,0.00,0.00,-16.00,0.00,"-168,469.60",1.00,0.00
25%,2.00,1.25,"13,964.00",4.35,123.00,239.99,6.00,0.00,0.00,4.12,2.42,6.47,100.00,130.00
50%,5.00,1.95,"15,228.00",11.70,228.00,387.39,14.00,0.00,0.00,7.99,2.86,14.44,198.00,372.00
75%,12.00,3.75,"16,788.00",19.50,413.00,681.59,33.00,0.00,0.00,12.39,3.61,20.41,370.00,525.00
max,"80,995.00","38,970.00","18,287.00","168,469.60","87,167.00","168,469.60",510.00,112.00,100.00,"12,540.00","25,111.09","168,469.60","2,557.00",738.00


    ^ Il y a des valeurs suspectes sur le min et le max de Quantity et de Price

Quantity

In [30]:
df[['Quantity', 'Description', 'Price', 'InvoiceDate']].sort_values('Quantity', ascending=False)

,Quantity,Description,Price,InvoiceDate
1031554,80995,"PAPER CRAFT , LITTLE BIRDIE",2.08,2011-12-09 09:15:00
557400,74215,MEDIUM CERAMIC TOP STORAGE JAR,1.04,2011-01-18 10:01:00
89849,19152,BLACK AND WHITE PAISLEY FLOWER MUG,0.10,2010-02-15 11:57:00
125791,12960,SET/6 WOODLAND PAPER PLATES,0.10,2010-03-17 13:09:00
125789,12960,SET/6 STRAWBERRY PAPER CUPS,0.10,2010-03-17 13:09:00
...,...,...,...,...
355457,-7008,SET/6 FRUIT SALAD PAPER PLATES,0.13,2010-09-28 11:02:00
355456,-7128,SET/6 FRUIT SALAD PAPER CUPS,0.08,2010-09-28 11:02:00
500620,-9360,ROTATING SILVER ANGELS T-LIGHT HLDR,0.03,2010-12-02 14:23:00
557405,-74215,MEDIUM CERAMIC TOP STORAGE JAR,1.04,2011-01-18 10:17:00


    ^ Les valeurs max et min en quantité sont des erreurs de commandes qui ont été annulées ensuite.

In [31]:
# Suppression des valeurs extrêmes pour la quantité
df = df[(df.Quantity < 20_000) & (df.Quantity > -10_000)]

Price

In [32]:
df[['Quantity', 'Description', 'Price']].sort_values('Price', ascending=False)

,Quantity,Description,Price
717220,-1,Manual,"38,970.00"
238926,-1,Manual,"25,111.09"
133500,-1,Manual,"10,953.50"
133499,1,Manual,"10,953.50"
133498,-1,Manual,"10,953.50"
...,...,...,...
39637,12,ANTIQUE GLASS HEART DECORATION,0.00
164238,1,AIRLINE BAG VINTAGE JET SET WHITE,0.00
946873,24,PAPER BUNTING VINTAGE PAISLEY,0.00
946872,20,SET OF 9 BLACK SKULL BALLOONS,0.00


    ^ Les valeurs extrêmes de prix sont liées surcoût bancaires ou à Amazon, on va donc supprimer au-delà d'un certain palier

In [33]:
df = df[(df.Price > -10_000) & (df.Price < 10_000)]

StockCode

In [34]:
nombre_de_lignes_adjust = df[df.StockCode == 'ADJUST'].shape[0] # <- compte le nombre de lignes dont le stockcode est adjust (=67)
df[df.StockCode == 'ADJUST'].sample(2)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total,Order_Quantity,...,Cancelled,Order_count,Cancelled_order_count,Percent_cancelled,Average_Quantity,Average_Price,Average_Basket,StockCode_count,Recency,Flag
70260,C495751,ADJUST,Adjustment by john on 26/01/2010 16,-1,2010-01-26 16:28:00,26.19,"12,858.00",United Kingdom,-26.19,-1,...,True,2,1.00,50.00,1.32,3.89,-26.19,37,681,🇬🇧
70288,495782,ADJUST,Adjustment by john on 26/01/2010 17,1,2010-01-26 17:10:00,139.71,"14,156.00",EIRE,139.71,1,...,False,202,0.00,0.00,39.40,8.84,139.71,1446,681,🇮🇪


    ^ Ces 67 lignes ne sont pas des commandes mais des réajustements

In [35]:
#Suppression des lignes dont le code de stock est 'ADJUST'
df = df[df.StockCode != 'ADJUST']

Unspecified country

In [36]:
#Vérifications du nombre de lignes où le pays n'est pas specifié
df[df.Country == 'Unspecified'].shape[0] # = 751

#Suppression de ces lignes
df = df[df.Country != 'Unspecified']

In [37]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Diviser le DataFrame en deux parties équilibrées
df_2009_2010 = df[df['InvoiceDate'].dt.year.isin([2009, 2010])]
df_2011 = df[df['InvoiceDate'].dt.year == 2011]

# Enregistrer les deux parties dans des fichiers CSV distincts
df_2009_2010.to_csv('data/silver_2009_2010_cleaned.csv', index=False)
df_2011.to_csv('data/silver_2011_cleaned.csv', index=False)